In [1]:
# %pip install pdf2image
# %pip install opencv-python pillow
# %pip install pytesseract

In [45]:
from pdf2image import convert_from_path
import pytesseract
import cv2
from pytesseract import Output
import pandas as pd

In [47]:
def paraBreakdown(folderName,pageNum):
    img = cv2.imread(f'{folderName}/Page_{pageNum}.jpg')
    custom_config = r'--psm 3'
    d = pytesseract.image_to_data(img,config=custom_config, output_type=Output.DICT)
    # print(d.keys())
    df = pd.DataFrame(d)
    #Grouping the text by their block numbers
    processedDf = df.groupby(['block_num']).agg(
        {
            'left':min,  
            'top': min,  
            'width': max,
            'height': max,
            'conf': 'mean',
            'text': (lambda x: ' '.join(x))
        }
    )
    # # Creating a visualization that showcases the boxes 
    # for i in range(len(grouped_df)):
    #     # print(grouped_df.loc[i,'left'])
    # #     if int(d['conf'][i]) > 60:
    #     (x, y, w, h) = (grouped_df.loc[i,'left'], grouped_df.loc[i,'top'], grouped_df.loc[i,'width'], grouped_df.loc[i,'height'])
    #     img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)


cv2.imshow('img', img)
cv2.waitKey(5000)
    grouped_df = processedDf.reset_index()
    grouped_df['Page'] = pageNum
    # grouped_df.to_csv(f'./Page_{pageNum}paraBlocksTrial.csv')

    # display(grouped_df)
    return grouped_df

In [48]:
#Converting pdf pages into individual images
def convertPDFtoImages(pdfPath,pdfFolderToStoreImages,outputFilePath):
    pdfs = pdfPath #"./Articles/amz_letters.pdf"
    pages = convert_from_path(pdfs, 350)
    print('Ingested all the pages from the pdf. Moving on to process them next ==>')
    df = pd.DataFrame()
    i = 1
    for page in pages:
        image_name = "Page_" + str(i) + ".jpg"  
        page.save(f'{pdfFolderToStoreImages}/{image_name}', "JPEG")
        print(f'Doing iteration number {i}')
        res = paraBreakdown(pdfFolderToStoreImages,i)
        # display(res)
        df=pd.concat([df,res], ignore_index=True)
        i = i+1
    display(df)
    df.to_csv(f'{outputFilePath}')

In [49]:
convertPDFtoImages('./Articles/amz_letters.pdf','AMZ_Letters','AMZ_Letters/fullAMZParaBlocks.csv')

Ingested all the pages from the pdf. Moving on to process them next ==>
Doing iteration number 1
Doing iteration number 2
Doing iteration number 3
Doing iteration number 4
Doing iteration number 5
Doing iteration number 6
Doing iteration number 7
Doing iteration number 8
Doing iteration number 9
Doing iteration number 10
Doing iteration number 11
Doing iteration number 12
Doing iteration number 13
Doing iteration number 14
Doing iteration number 15
Doing iteration number 16
Doing iteration number 17
Doing iteration number 18
Doing iteration number 19
Doing iteration number 20
Doing iteration number 21
Doing iteration number 22
Doing iteration number 23
Doing iteration number 24
Doing iteration number 25
Doing iteration number 26
Doing iteration number 27
Doing iteration number 28
Doing iteration number 29
Doing iteration number 30
Doing iteration number 31
Doing iteration number 32
Doing iteration number 33
Doing iteration number 34
Doing iteration number 35
Doing iteration number 36
D

,block_num,left,top,width,height,conf,text,Page
0,0,0,0,2975,3850,-1.000000,,1
1,1,2383,99,127,38,23.000000,1997,1
2,2,996,526,990,116,21.500000,amaZon.com,1
3,3,1010,660,978,48,23.000000,,1
4,4,439,849,475,42,47.500000,To our shareholders:,1
...,...,...,...,...,...,...,...,...
934,1,330,309,2191,161,86.892857,of high standards. I guarantee you that And...,88
935,2,330,522,2142,104,84.060606,"To all of you: be kind, be original, create...",88
936,3,332,678,187,45,23.250000,"Sincerely,",88
937,4,330,1090,312,45,45.166667,Jeffrey P. Bezos,88
